In [1]:
import os
import polars as pl

In [2]:
df = pl.read_ndjson("data_raw/fdb/*.*")

In [3]:
df.sample(10)

kanal,datum,cas,id,nazev,extra,noticka
str,str,str,str,str,str,str
"""Cinemax2""","""06.06.2017""","""22:10""","""./../film/liberace-behind-the-…","""Liberace!""","""10.06.2022 v 15:10 (HBO)20.06.…","""Další díl/opakování:10.06.2022…"
"""PLUS""","""10.06.2011""","""12:30""","""./../serial/m-a-s-h-m-a-s-h/41…","""M.A.S.H.""","""20.01.2025 v 05:20 (Prima MAX)…","""Další díl/opakování:20.01.2025…"
"""Nova Cinema""","""20.12.2013""","""14:40""","""./../serial/odlozene-pripady-c…","""Odložené případy""","""25.12.2024 v 02:10 (Nova Gold)…","""Další díl/opakování:25.12.2024…"
"""Prima""","""09.11.2014""","""13:50""","""./../serial/big-ben-der-bulle-…","""Big Ben""","""07.10.2024 v 16:00 (Prima Krim…","""Další díl/opakování:07.10.2024…"
"""Prima LOVE""","""24.11.2014""","""15:35""","""./../serial/policie-hamburk-no…","""Policie Hamburk""","""18.01.2025 v 21:05 (Prima STAR…","""Další díl/opakování:18.01.2025…"
"""ČT2""","""14.10.2012""","""16:00""",null,"""Cesty víry: Milujte život""",null,"""Magazín. Odkaz Josefa Navrkala…"
"""Markíza""","""27.08.2022""","""06:55""","""./../serial/dobrodruzstvi-toma…","""Príbehy Toma a Jerryho""","""21.10.2023 v 06:10 (NOVA)22.10…","""Další díl/opakování:21.10.2023…"
"""ČT 3""","""04.01.2021""","""19:00""","""./../film/udalosti/67746""","""Události""","""17.04.2025 v 18:55 (ČT1)17.04.…","""Další díl/opakování:17.04.2025…"
"""JOJ""","""22.09.2016""","""22:00""",null,"""Návštevníci""",null,"""Cesta do minulosti. Reality sh…"


In [4]:
slovenske = ["Markíza","JOJ","JOJ+","PLUS","PLUS HD"]

In [19]:
df_ciste = df.filter(
    ~pl.col("kanal").is_in(slovenske)
).with_columns(
    (pl.col('datum') + pl.lit(' ') + pl.col('cas')).str.to_datetime().alias('datum')
).rename({'id':'url'}
).select(
    pl.col(['kanal','datum','nazev','extra','noticka','url'])
).with_columns(
    pl.col('kanal').cast(pl.Categorical)
).sort(
    by="datum"
).unique(
    subset=['kanal','datum','nazev'], keep='first'
).with_columns(pl.col("url").str.replace('./../',''))

C:\Users\micha\AppData\Local\Temp\ipykernel_17276\941640856.py:8: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  ).with_columns(


In [39]:
df_ciste.sample(10)

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2013-04-23 12:20:00,"""BBV, Předpověď počasí, Událost…",null,null,null
"""Prima LOVE""",2016-12-26 18:40:00,"""Ano, šéfe s Gordonem Ramsaym: …","""20.01.2025 v 15:15 (JOJ+)21.01…","""Další díl/opakování:20.01.2025…","""film/gordon-ramsay-nocni-mury-…"
"""Prima LOVE""",2012-11-01 13:25:00,"""Velmi křehké vztahy""","""19.01.2025 v 13:20 (Prima STAR…","""Další díl/opakování:19.01.2025…","""serial/velmi-krehke-vztahy/452…"
"""TV Barrandov""",2018-07-06 02:25:00,"""Na plac!""","""18.01.2025 v 02:20 (TV Barrand…","""Další díl/opakování:18.01.2025…","""film/na-plac/103088"""
"""VOYO""",2019-11-24 18:20:00,"""Záhada hlavolamu""","""27.12.2024 v 18:00 (CS Film)30…","""Další díl/opakování:27.12.2024…","""film/zahada-hlavolamu/23457"""
"""Cinemax""",2015-06-14 11:05:00,"""Velký skok""","""12.02.2016 v 09:15 (Cinemax)13…","""Další díl/opakování:12.02.2016…","""film/velky-skok-the-big-leap/1…"
"""ČT1""",2009-10-21 18:45:00,"""Večerníček""","""31.01.2025 v 18:45 (ČT :D / ČT…","""Další díl/opakování:31.01.2025…","""film/vecernicek/71245"""
"""ČT1""",2024-03-12 09:00:00,"""Hotel Portofino""","""II""",null,null
"""Cinemax""",2023-11-12 04:20:00,"""Jsme živé bytosti""","""Drama Čína/It./USA (2021). Syn…",null,null


In [23]:
df_ciste.shape

(2000449, 6)

In [7]:
kam = "data"
if not os.path.exists(kam):
    os.makedirs(kam)

In [25]:
df_ciste.write_parquet(os.path.join(kam,'fdb.parquet'))